In [1]:
import json
import numpy as np
import pandas as pd
import catboost as cb
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

In [2]:
features_dir = '../features/'
data = pd.read_csv(features_dir + 'train_test.csv')
cat_features = eval(open(features_dir + 'cat_cols').read())
cat_features.extend(eval(open(features_dir + 'cat_cols_expand').read()))
cat_features = list(set(cat_features))
cat_features = [i for i in cat_features if i in data.columns]
text_features = ['opscope']
for i in cat_features:
    data[i] = data[i].astype('str')

In [3]:
def training(X_train, y_train, test):
    cat = CatBoostClassifier(iterations=500, silent=True)
    cat.fit(X_train, y=y_train,
        cat_features=cat_features,
        # text_features=text_features, 
        eval_set=(X_test, y_test)
    )
    return cat

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,1:-1], data.iloc[:,-1], test_size=0.2)
# cat = training(X_train, y_train, (X_test, y_test))
# y_pred = pd.Series(cat.predict_proba(X_test)[:,1]).apply(lambda x:1 if x > 0.5 else 0)
# print(classification_report(y_test, y_pred))

In [5]:
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
skf = StratifiedKFold()
report = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    cat = training(X_train, y_train, (X_test, y_test))
    y_pred = pd.Series(cat.predict_proba(X_test)[:,1]).apply(lambda x:1 if x > 0.5 else 0)
    report.append(classification_report(y_test, y_pred))
    print(report[-1], '\n')

              precision    recall  f1-score   support

         0.0       0.99      0.98      0.99      2777
         1.0       0.79      0.89      0.84       196

    accuracy                           0.98      2973
   macro avg       0.89      0.94      0.91      2973
weighted avg       0.98      0.98      0.98      2973
 

              precision    recall  f1-score   support

         0.0       0.99      0.98      0.99      2777
         1.0       0.79      0.88      0.83       196

    accuracy                           0.98      2973
   macro avg       0.89      0.93      0.91      2973
weighted avg       0.98      0.98      0.98      2973
 

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      2777
         1.0       0.82      0.87      0.84       196

    accuracy                           0.98      2973
   macro avg       0.90      0.93      0.92      2973
weighted avg       0.98      0.98      0.98      2973
 

             

In [6]:
cat = CatBoostClassifier(iterations=500, silent=True)
cat.fit(data.iloc[:,1:-1], data.iloc[:,-1], cat_features=cat_features)

In [7]:
features_dir = '../features/'
data = pd.read_csv(features_dir + 'evaluate.csv')
for i in cat_features:
    data[i] = data[i].astype('str')
evaluate = pd.DataFrame(data.id)
evaluate['score'] = pd.DataFrame(cat.predict_proba(data.iloc[:,1:])[:,1])

In [8]:
submit = pd.read_csv('../entprise_evaluate.csv')
submit = submit.merge(evaluate, on='id')
# submit.to_csv('../submit.csv', index=False)